## 02. Train the model. Gets kmeans dictionary

In [ ]:
sc.version
val conf = sc.hadoopConfiguration
val fs = org.apache.hadoop.fs.FileSystem.get(conf)
val hdfshome = fs.getHomeDirectory
println(sc.version)
println(hdfshome)
val files = fs.listFiles(hdfshome, false)

In [ ]:
val rootDir = "/user/fsainz/data/out"
val files = fs.listFiles(new org.apache.hadoop.fs.Path(rootDir), false)


In [ ]:
//fs.exists(new org.apache.hadoop.fs.Path("hdfs://spark-clu-m/user/fsainz/data/in/a02.dat"))
fs.exists(new org.apache.hadoop.fs.Path(rootDir))

## Windowing. Split in Signal in windows


In [ ]:

//val inFileName = "105s1"   //mit-arritmia
//val wSize = 120;              // sliding windows size   
//val step = wSize/10;          // window overriding step  (10% of windows size)        

val inFileName = "222"   //mit-arritmia
val wSize = 120;              // sliding windows size   
val step = wSize/10;          // window overriding step  (10% of windows size)    

//val inFileName = "a02"     //apnea
//val wSize = 40;              // sliding windows size   
//val step = wSize/10;   

val scale = 1/200.0;

In [ ]:

import java.io._                                                        
import scala.collection.mutable.ListBuffer   //bE CAREFULL With java imports, better before scala.


val inputPath =  "/home/fsainz/notebooks/data/arritmia/" //local filesystem
val fileName = inputPath + inFileName + ".dat"
val hdfsOutPath = "/user/fsainz/data/out/"
val outFileName = hdfsOutPath + "dict-" + inFileName + "_W" + wSize + "-"+step +".LE.model"


println("InFile: " + fileName)
println("Ouput Model: " + outFileName)

## Use a circular buffer to read data from file, and obtain sliding windows
Take into account that arithmia db file has two signal, and we use only one of then


val TRsamples = (maxSlidingW * .2).toInt;       // number of samples to train the model## Read Apnea DB file

###  Apnea DB

In [ ]:



// Now start reading local file   TRaining file is part InFile training and run

var inFile =  new File(fileName)
var InFileStream = new FileInputStream( inFile )
val input : DataInputStream  = new DataInputStream(InFileStream)

val maxSlidingW= (inFile.length / 2) / step    // each data in the file is two byte long
val TRsamples = (maxSlidingW * .2).toInt;       // number of samples to train the model

// val nSamples =  (inFile.length() / 2)   // We are going to read short: 2 bytes..


val buffer = new Array[Double](wSize)
var windows = scala.collection.mutable.ListBuffer.empty[Array[Double]]

var p_buff = 0
var available = 0
var dataWindow = new Array[Double](wSize)
var readed = 0

// Read data and create slinding windows. Every STEPS reads create a new windows
for (  w  <- 1 to TRsamples){
  dataWindow = new Array[Double](wSize)  // new object is need to append to the list!!
  var newWin:Boolean = false
  while( ! newWin) { 
    // .dat files are little endian 
    val value = java.lang.Short.reverseBytes(input.readShort())
    
    print ("%x ".format(value), value, "\n")
    buffer( p_buff ) = (value.toDouble * scale) 
    available += 1
    p_buff +=1
    if ( p_buff == wSize )  // buffer ends
      p_buff = 0  
      
    if (available == wSize) {    // There is a window ready
      var j = 0
      // buffer is circular buffers last readings firts.
      for( i <- p_buff until wSize){
         dataWindow(j) = buffer(i)
         j +=1
      }
      for( i <- 0 until p_buff ){
         dataWindow(j) = buffer(i)
         j +=1
      }
      //dataWindow.foreach(print)
      //println("|")
      newWin = true
      available = wSize-step   // ready for read wSize displacement offset 
    }
  }
  windows.append(dataWindow  )  
}
input.close()

## Read Arrithmia db files

In [ ]:


// Now start reading local file   TRaining file is part InFile training and run

var inFile =  new File(fileName)
var InFileStream = new FileInputStream( inFile )
val input : DataInputStream  = new DataInputStream(InFileStream)

val fileSamples = inFile.length / 3     // arithmia files records have two signal of 12 bytes length 3bytes
val maxSlidingW= fileSamples / step    // each data in the file is two byte long
val TRsamples = (maxSlidingW * .2).toInt;       // number of samples to train the model

val u4mask = 0xf0
val l4mask = 0x0f

val buffer = new Array[Double](wSize)
var windows = scala.collection.mutable.ListBuffer.empty[Array[Double]]

var p_buff = 0
var available = 0
var dataWindow = new Array[Double](wSize)
var readed = 0
val r = new Array[Byte](3)

// Read data and create slinding windows. Every STEPS reads create a new windows
for (  w  <- 1 to TRsamples){
  dataWindow = new Array[Double](wSize)  // new object is need to append to the list!!
  var newWin:Boolean = false
  while( ! newWin) { 
    // .dat files are little endian 
    input.read(r)
    val s1 = ( ((r(1) & l4mask)) << 8 ) + (r(0) & 0xff)
    val s2 = ( (r(1) & u4mask )  << 4 ) + (r(2) & 0xff)  
    readed +=1   
    print ("%x ".format(s1), "%x".format(s2), "\n")
      
    buffer( p_buff ) = (s1.toDouble * scale) 
    available += 1
    p_buff +=1
    if ( p_buff == wSize )  // buffer ends
      p_buff = 0  
      
    if (available == wSize) {    // There is a window ready
      var j = 0
      // buffer is circular buffers last readings firts.
      for( i <- p_buff until wSize){
         dataWindow(j) = buffer(i)
         j +=1
      }
      for( i <- 0 until p_buff ){
         dataWindow(j) = buffer(i)
         j +=1
      }
      //dataWindow.foreach(print)
      //println("|")
      newWin = true
      available = wSize-step   // ready for read wSize displacement offset 
    }
  }
  windows.append(dataWindow  )  
}
input.close()
print(readed)

## Defining windowing function

In [ ]:

var windowingF  =  new Array[Double](wSize)   //differs of Array[Double](WINDOW) uajj
for ( i <- 0 until wSize) {
    val y = Math.sin(Math.PI * i / (wSize - 1.0));
          windowingF(i)=  y * y
 }


In [ ]:
// spark 2.1 mllib
import org.apache.spark.mllib.clustering.{KMeans, KMeansModel}
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.regression.LabeledPoint 

## Normalize and Apply Windowing function each segment before clustering
Nomalizar $$ (X-mean(x))/Norm(x)$$


### Following should work with no problem but it not!!. but It works in spark-shell.¶
So it seems to be a problems of Apache Toree jupyter kernel.

In [ ]:
val ECGRdd = sc.parallelize(windows)
val ECGnormRdd = ECGRdd.map( x => { val n= x.length; val s = x.reduce(_+_); (x,s/n) }).   // get avg
                        map( x => (Vectors.dense(x._1), x._1.map(i => i - x._2))).                    // susbtracs avg
                        map( x => { val nr = Vectors.norm(x._1,2); x._2.map( i => i/nr) })

val ECGwnRdd = ECGnormRdd.map( x => ((x,windowingF).zipped.map(_*_)))
ECGwnRdd.first


### So make a workaround of save windows to files..

In [ ]:

// FileWriter
import org.apache.hadoop.fs.Path;
val csvFileName = hdfsOutPath + "wslices-f"+ inFileName + "-w" +wSize +"-s"+step + ".csv"

// write on hdfs as normal file
print("SAving: " +csvFileName )
val output = fs.create(new Path(csvFileName))

println("writing windows files: "+ csvFileName)
//val bw = new BufferedWriter(new FileWriter(output))
for ( w <- windows ){
    var line = ""
   w.foreach( x => line +=  x.toString + "," )
   output.write( (line.substring(0,line.length-1) + "\n").getBytes())
} 
output.close()




### And read in spark...
It seem to be necesary to restart the jupyter kernel before progress..

In [ ]:
val csvFileName = hdfsOutPath + "wslices-f"+ inFileName + "-w" +wSize +"-s"+step + ".csv"

In [ ]:
print("loading windows " + csvFileName )
val ECGRdd = sc.textFile(csvFileName).map(x => x.split(",").map(_.toDouble) )
ECGRdd.count

In [ ]:
val ECGnormRdd = ECGRdd.map( x => { val n= x.length; val s = x.reduce(_+_); (x,s/n) }).   // get avg
                        map( x => (Vectors.dense(x._1), x._1.map(i => i - x._2))).                    // susbtracs avg
                        map( x => { val nr = Vectors.norm(x._1,2); x._2.map( i => i/nr) })

ECGnormRdd.first.length

In [ ]:
val ECGwnRdd = ECGnormRdd.map( x => Vectors.dense( ((x,windowingF).zipped.map(_*_)) ))
ECGwnRdd.first


In [ ]:
// save tapped windows for documntation images.

import org.apache.hadoop.fs.Path;
val tapWin = ECGnormRdd.collect()
// write on hdfs as normal fileECGwnRdd
val csvtapWindows = hdfsOutPath + "TAPslices-"+ inFileName+ wSize+"-"+step + ".csv"
print("SAving: " +csvtapWindows )
val output = fs.create(new Path(csvtapWindows))

println("writing windows files: "+ csvtapWindows)
//val bw = new BufferedWriter(new FileWriter(output))
for ( w <- tapWin ){
    var line = ""
   w.toArray.foreach( x => line +=  x.toString + "," )
   
    //output.write( (line.substring(0,line.length-1) + "\n").getBytes())

} 
output.close()


In [ ]:
    ECGwnRdd.cache()  // important to kmeans time
    val numIterations = 20// we want to assure a global mininum with 20 it seem not to be reached
    val numClusters = ( ECGRdd.count * 0.10).toInt
    val clusters = KMeans.train(ECGwnRdd, numClusters, numIterations)

In [ ]:
clusters

In [ ]:
clusters.save(sc, outFileName)

println(outFileName + " SAVED")